## 일단 라이브러리를 설치합시다.

- 혹시 이런 것들이 필요할지도...

python -m pip install --upgrade pip

python -m pip --version


- 그리고 이건 왠만하면 합시다.

python -m pip install --user virtualenv

python -m venv myenv

.\myenv\Scripts\activate


- 이건 필수입니다.

pip install djitellopy2

pip install opencv-python

## 일단, 기본적인 동작이 되는지 확인합니다.
### 1) 비행이 되는지 확인

In [ ]:
from djitellopy import Tello
import time

# Connect to Tello
tello = Tello()
tello.connect()

# Takeoff
tello.takeoff()
time.sleep(2)

# Move forward
tello.move_forward(30)
time.sleep(2)

# Rotate clockwise
tello.rotate_clockwise(45)
time.sleep(2)

# Land
tello.land()


### 2) 화면은 오는지 확인

In [ ]:
from djitellopy import Tello
import cv2

# Connect to Tello
tello = Tello()
tello.connect()

# Start video stream
tello.streamon()

# OpenCV window to display video feed
cv2.namedWindow("Tello Video Feed")

while True:
    # Get frame from video stream
    frame = tello.get_frame_read().frame

    # Display the frame
    cv2.imshow("Tello Video Feed", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Stop video stream
tello.streamoff()

# Close OpenCV window
cv2.destroyAllWindows()


### 3) 배터리 정보를 읽어봅시다.

In [ ]:
from djitellopy import Tello
import time

# Connect to Tello
tello = Tello()
tello.connect()

# Start video stream
tello.streamon()

# Display battery level periodically
while True:
    # Get battery level
    battery_level = tello.get_battery()

    # Display battery level
    print(f"Battery Level: {battery_level}%")

    # Wait for 5 seconds
    time.sleep(5)


## 이제 좀 더 그럴듯하게 해볼까요?
### 1) 비행하면서 화면을 보여주면서 배터리 잔량도 동시에 확인합시다.

In [ ]:
from djitellopy import Tello
import cv2
import time
import threading

def display_video(tello):
    while True:
        frame = tello.get_frame_read().frame
        cv2.imshow("Tello Video Feed", frame)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        time.sleep(0.1)

def check_battery(tello):
    while True:
        battery_level = tello.get_battery()
        print(f"Battery Level: {battery_level}%")
        time.sleep(5)

def main():
    # Connect to Tello
    tello = Tello()
    tello.connect()

    # Start video stream
    tello.streamon()

    # Create and start threads for video display and battery check
    video_thread = threading.Thread(target=display_video, args=(tello,), daemon=True)
    battery_thread = threading.Thread(target=check_battery, args=(tello,), daemon=True)


    video_thread.start()
    battery_thread.start()

    # Takeoff
    tello.takeoff()
    time.sleep(2)

    # Rotate clockwise
    tello.rotate_clockwise(360)
    time.sleep(2)

    # Land
    tello.land()

    # Stop video stream
    tello.streamoff()

if __name__ == "__main__":
    import threading

    main()

### 2) 이제는 조종을 해볼까요?

In [ ]:
import cv2
from djitellopy import Tello
import time
import threading

def check_battery(tello):
    while True:
        battery_level = tello.get_battery()
        print(f"Battery Level: {battery_level}%")
        time.sleep(5)

# Connect to Tello
tello = Tello()
tello.connect()

# Create and start threads for video display and battery check
battery_thread = threading.Thread(target=check_battery, args=(tello,), daemon=True)
battery_thread.start()

# Start video stream
tello.streamon()

# Takeoff
tello.takeoff()
tello.move_up(50)  # Adjust the height as needed
cv2.waitKey(2)  # Give some time for the drone to stabilize

# Main loop for face tracking
while True:
    # Get frame from video stream
    frame = tello.get_frame_read().frame

    # Display the frame
    cv2.imshow("Tello Video Feed", frame)

    # Capture key presses
    key = cv2.waitKey(1) & 0xFF

    # Drone control based on key presses
    if key == ord('q'):  # Quit the program
        break
    elif key == ord('w'):  # Move up
        tello.move_up(20)
    elif key == ord('s'):  # Move down
        tello.move_down(20)
    elif key == ord('a'):  # Move left
        tello.move_left(20)
    elif key == ord('d'):  # Move right
        tello.move_right(20)

# Land
tello.land()

# Stop video stream
tello.streamoff()

# Close OpenCV window
cv2.destroyAllWindows()

### 3) Face detection은 되는가?

In [ ]:
import cv2
from djitellopy import Tello
import time
import threading

def check_battery(tello):
    while True:
        battery_level = tello.get_battery()
        print(f"Battery Level: {battery_level}%")
        time.sleep(5)

# Connect to Tello
tello = Tello()
tello.connect()

# Create and start threads for video display and battery check
battery_thread = threading.Thread(target=check_battery, args=(tello,), daemon=True)
battery_thread.start()

# Start video stream
tello.streamon()

# Takeoff
tello.takeoff()
tello.move_up(50)  # Adjust the height as needed
cv2.waitKey(2)  # Give some time for the drone to stabilize

# Initialize face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


# Main loop for face tracking
while True:
    # Get frame from video stream
    frame = tello.get_frame_read().frame

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # If faces are detected, track the first face
    if len(faces) > 0:
        face = faces[0]
        cv2.rectangle(frame, (face[0], face[1]), (face[0] + face[2], face[1] + face[3]), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Tello Video Feed", frame)

    # Capture key presses
    key = cv2.waitKey(1) & 0xFF

    # Drone control based on key presses
    if key == ord('q'):  # Quit the program
        break
    elif key == ord('w'):  # Move up
        tello.move_up(20)
    elif key == ord('s'):  # Move down
        tello.move_down(20)
    elif key == ord('a'):  # Move left
        tello.move_left(20)
    elif key == ord('d'):  # Move right
        tello.move_right(20)

# Land
tello.land()

# Stop video stream
tello.streamoff()

# Close OpenCV window
cv2.destroyAllWindows()

## 자, 그럼 이제 object detection을 해볼까요?
### 1) 일단 일반적인 object detection
#### - 참조할 사이트는 https://github.com/AlexeyAB/darknet
#### - coco.names, yolov4-tiny.weights, yolov4-tiny.cfg 파일을 받아 줍니다.

In [ ]:
import cv2
import numpy as np

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4/yolov4-tiny.weights", "yolov4/yolov4-tiny.cfg")
classes = []
with open("yolov4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

# Open a video capture
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or specify the path to a video file

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Object detection with YOLOv4
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Process the results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected, get coordinates
                center_x, center_y, w, h = (detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype('int')
                
                # Calculate top-left corner
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw bounding box and label on the frame
                color = (0, 255, 0)  # Green
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the frame
    cv2.imshow("YOLOv4 Object Detection", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close OpenCV window
cap.release()
cv2.destroyAllWindows()


### 2) 이제 tello와 결합해볼까요?

In [ ]:
import cv2
import numpy as np
from djitellopy import Tello
import time
import threading

def check_battery(tello):
    while True:
        battery_level = tello.get_battery()
        print(f"Battery Level: {battery_level}%")
        time.sleep(5)

def object_detection(frame, net, classes):
    # Object detection with YOLOv4
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Process the results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected, get coordinates
                center_x, center_y, w, h = (detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype('int')
                
                # Calculate top-left corner
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw bounding box and label on the frame
                color = (0, 255, 0)  # Green
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

# Connect to Tello
tello = Tello()
tello.connect()

# Create and start threads for video display and battery check
battery_thread = threading.Thread(target=check_battery, args=(tello,), daemon=True)
battery_thread.start()

# Start video stream
tello.streamon()

# Takeoff
tello.takeoff()
tello.move_up(50)  # Adjust the height as needed
cv2.waitKey(2)  # Give some time for the drone to stabilize

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4/yolov4-tiny.weights", "yolov4/yolov4-tiny.cfg")
classes = []
with open("yolov4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

# Main loop for face tracking
while True:
    # Get frame from video stream
    frame = tello.get_frame_read().frame
    frame = object_detection(frame, net, classes)

    # Display the frame
    cv2.imshow("Tello Video Feed", frame)

    # Capture key presses
    key = cv2.waitKey(1) & 0xFF

    # Drone control based on key presses
    if key == ord('q'):  # Quit the program
        break
    elif key == ord('w'):  # Move up
        tello.move_up(20)
    elif key == ord('s'):  # Move down
        tello.move_down(20)
    elif key == ord('a'):  # Move left
        tello.move_left(20)
    elif key == ord('d'):  # Move right
        tello.move_right(20)

# Land
tello.land()

# Stop video stream
tello.streamoff()

# Close OpenCV window
cv2.destroyAllWindows()

### 3) 필요한 detection 정보를 빼와 봅시다.

In [ ]:
import cv2
import numpy as np
from djitellopy import Tello
import time
import threading

def check_battery(tello):
    while True:
        battery_level = tello.get_battery()
        print(f"Battery Level: {battery_level}%")
        time.sleep(5)

def object_detection(frame, net, classes):
    # Object detection with YOLOv4
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Process the results
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected, get coordinates
                center_x, center_y, w, h = (detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])).astype('int')
                
                # Calculate top-left corner
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw bounding box and label on the frame
                color = (0, 255, 0)  # Green
                label = f"{classes[class_id]}: {confidence:.2f}"
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame

# Connect to Tello
tello = Tello()
tello.connect()

# Create and start threads for video display and battery check
battery_thread = threading.Thread(target=check_battery, args=(tello,), daemon=True)
battery_thread.start()

# Start video stream
tello.streamon()

# Takeoff
tello.takeoff()
tello.move_up(50)  # Adjust the height as needed
cv2.waitKey(2)  # Give some time for the drone to stabilize

# Load YOLOv4 model
net = cv2.dnn.readNet("yolov4/yolov4-tiny.weights", "yolov4/yolov4-tiny.cfg")
classes = []
with open("yolov4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

# Main loop for face tracking
while True:
    # Get frame from video stream
    frame = tello.get_frame_read().frame
    frame = object_detection(frame, net, classes)

    # Display the frame
    cv2.imshow("Tello Video Feed", frame)

    # Capture key presses
    key = cv2.waitKey(1) & 0xFF

    # Drone control based on key presses
    if key == ord('q'):  # Quit the program
        break
    elif key == ord('w'):  # Move up
        tello.move_up(20)
    elif key == ord('s'):  # Move down
        tello.move_down(20)
    elif key == ord('a'):  # Move left
        tello.move_left(20)
    elif key == ord('d'):  # Move right
        tello.move_right(20)

# Land
tello.land()

# Stop video stream
tello.streamoff()

# Close OpenCV window
cv2.destroyAllWindows()

## 자, 이제는 나만의 모델을 만들어볼까요?

1. 사진을 확보해야 합니다.
- 인식하고자 하는 사물의 특성을 잘 파악해야 합니다.
- 배경을 다양하게 하는게 중요합니다. (배경이 다양해야 인식하고자 하는 제품만의 특징을 잘 뽑아낼 수 있습니다.)
- 충분한 양의 사진을 준비하는 게 중요합니다. (사물당 최소 300장 이상)

2. 라벨링을 해야합니다.
    * Roboflow 서비스를 이용해봅시다. (https://roboflow.com/)
    * 안내에 따라 가입하고 Signin 하세요. (Google id로 가입 가능)
    
    * 프로젝트를 만듭니다.<br>
    <img src="images/capture14.png" style="width:700px"><br>
    <img src="images/capture15.png" style="width:700px">

    * 파일들을 끌어다 놔서 upload 합니다.<br>
    <img src="images/capture16.png" style="width:700px">

    * 이제 각각의 파일을 열어서 annotation (혹은 라벨링)을 진행합니다. (엄청난 노가다... 그러나... 참을성 있게..)<br>
    <img src="images/capture17.png" style="width:700px">

3. 라벨링이 다 되면 이제 데이터셋을 export 합니다.
    * 이렇게 하는 겁니다.<br>
    <img src="images/capture18.png" style="width:700px">

    * 물론 이때 우리는 yolov8을 선택합니다.<br>
    <img src="images/capture19.png" style="width:700px">

    * 데이터 셋 파일을 다운로드 받거나, 다운로드 코드를 수행합니다.

3. 이제 다운 받은 이미지를 한 곳(images)에 모으고 구글 드라이브에 저장합시다.

    - 왜냐면 이제는 학습(train)을 위해서 Google colab을 활성화해야 하니까요.
    - Train을 위한 프로그램은 이걸 쓸겁니다. (https://colab.research.google.com/drive/1fdAyJwiApi_BT1iiKwqhcfzna2GKpylk)
    - 학습이 끝나면 모델은 여러분의 Google Drive에 저장됩니다. (yolov4_backup)